# Build model for different features
- HB Features
- Delineation Features
- HRV Features
- CNN Features

In [70]:
import sys

import pandas as pd
import numpy as np
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import seaborn as sns
import matplotlib.pyplot as plt

sns.set('talk')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
%matplotlib inline

params

In [2]:
train_size = 0.8

Read in features

In [3]:
path = '/home/rapwag01/eth/aml/aml-project/task2/data/'

X_train_hb = pd.read_csv(path+'first_run/features_heartbeat_train.csv', index_col=False).values
X_test_hb = pd.read_csv(path+'first_run/features_heartbeat_test.csv', index_col=False).values

X_train_cwt = pd.read_csv(path+'features_cwt_train.csv', index_col=False).values
X_test_cwt = pd.read_csv(path+'features_cwt_test.csv', index_col=False).values

X_train_delin = pd.read_csv(path+'first_run/features_delineation_train.csv', index_col=False).values
X_test_delin = pd.read_csv(path+'first_run/features_delineation_test.csv', index_col=False).values

X_train_hrv = pd.read_csv(path+'first_run/features_hrv_train.csv', index_col=False).values
X_test_hrv = pd.read_csv(path+'first_run/features_hrv_test.csv', index_col=False).values


Read in target

In [83]:
y_train = pd.read_csv('../../task2/y_train.csv').y.values.ravel()

In [84]:
y_train.shape

(5117,)

Join feature matrices
- Skip cwt for now

In [85]:
X_train_full = np.hstack([X_train_hb, X_train_delin, X_train_hrv])[:,1:] # drop first column which is index

In [86]:
X_test_full = np.hstack([X_test_hb, X_test_delin, X_test_hrv])[:,1:] # drop first column which is index

In [76]:
# don't need to apply train / test split if we use gridsearchcv
# X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, target, random_state=0, train_size=train_size, stratify=target)

remove samples with >80% NaNs

In [77]:
df_train_full = pd.DataFrame(X_train_full)

In [78]:
df_train_full.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988
0,-14.498234,-13.234579,-11.628894,-9.684386,-7.431202,-4.918017,-2.186781,0.719700,3.730826,6.763773,9.754217,12.654224,15.387219,17.858503,19.989768,21.740134,23.071220,23.946301,24.353872,24.313122,23.826370,22.872333,21.427582,19.466121,16.945076,13.789532,9.954404,5.429056,0.238137,-5.534010,-11.735787,-18.131505,-24.451479,-30.464007,-35.978735,-40.880341,-45.115909,-48.698300,-51.698581,-54.150485,-56.086609,-57.528309,-58.489907,-59.003520,-59.099691,-58.844840,-58.264232,-57.272099,-55.560516,-52.325388,-46.043368,-34.343886,-14.213665,17.301290,61.813321,118.367141,183.221865,249.319054,307.014239,346.379558,360.369182,347.130355,308.973094,252.583987,187.814469,124.789810,70.895187,28.894343,-1.227022,-21.555891,-35.419058,-46.058657,-55.581692,-64.479414,-72.558327,-79.542895,-85.247278,-89.626694,-92.800152,-95.014692,-96.535539,-97.528265,-98.079411,-98.268462,-98.182151,-97.877847,-97.381024,-96.717243,-95.922674,-95.021363,-94.015547,-92.901640,-91.685387,-90.388166,-89.011632,-87.539819,-85.974066,-84.309869,-82.534768,-80.618668,-78.546665,-76.313150,-73.882243,-71.248823,-68.420004,-65.416821,-62.280539,-59.067606,-55.863958,-52.705899,-49.611246,-46.578907,-43.560687,-40.476442,-37.218287,-33.699423,-29.858347,-25.650435,-21.043737,-16.026551,-10.591853,-4.728458,1.592870,8.417317,15.792974,23.810822,32.558446,42.093098,52.453073,63.630357,75.536966,87.945192,100.561262,113.104475,125.296269,136.883797,147.654250,157.488734,166.321037,174.048829,180.538305,185.606931,189.055862,190.660683,190.147849,187.280120,181.890234,173.972233,163.655362,151.162823,136.866472,121.242203,104.797544,87.898240,70.848631,53.972026,37.557659,21.829952,7.003641,-6.598808,-18.710091,-29.203930,-38.047099,-45.283864,-51.052402,-55.594070,-59.139776,-61.876320,-63.980914,-65.585412,-66.767905,-67.575868,-68.045530,-68.201873,-68.047691,-67.590102,-66.860754,-65.893511,-64.724872,-63.396068,-15.489147,-13.030004,-10.820615,-8.624940,-5.895799,-2.725400,-0.469552,2.266584,6.231460,10.126356,13.440836,15.515698,18.423118,20.660126,23.277018,24.091516,24.515854,25.127938,25

In [68]:
df_train_full['NaNs'] = df_train_full.isnull().sum(axis=1)

In [72]:
df_train_full['NaN_perc'] = df_train_full.NaNs/df_train_full.shape[1]

only one sample with almost all Nans -> remove

In [74]:
df_train_full['NaN_perc'].value_counts()

0.011111    4017
0.008081     742
0.015152     172
0.012121     113
0.089899      29
0.014141      21
0.016162       9
0.021212       8
0.013131       5
0.998990       1
Name: NaN_perc, dtype: int64

In [92]:
clf1 = XGBClassifier(learning_rate=0.05, n_estimators=300, max_depth=5)

clf2 = GradientBoostingClassifier(learning_rate=0.05, n_estimators=500, max_depth=7, 
                                 min_samples_split=60, min_samples_leaf=9, subsample=1.0,
                                 max_features=50, random_state=0)

param_grid = {
    'imputer': [SimpleImputer(strategy='constant', fill_value=0, add_indicator=True)],
    'scaler': [RobustScaler()],
    'selector': [SelectKBest()],
    'selector__k': [5, 10, 50],
    'pca': [KernelPCA()],
    'pca__n_components': [5, 10, 50],
    'clf': [clf1],
}

In [ ]:
#add VarianceThreshold, we got constant features (nan to zero)
#962  963  964  965  966  967  980  981 1951 1952 1953 1954 1955 1956

In [93]:
pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('scaler', RobustScaler()),
    ('threshold', VarianceThreshold()),
    ('selector', SelectKBest()),
    ('pca', KernelPCA()),
    ('clf', XGBClassifier())
    ])

In [94]:
grid = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='f1_micro', cv=5, verbose=10)

In [ ]:
grid.fit(X_train_full, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START clf=XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_delta_step=None, max_depth=5,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None), imputer=SimpleImputer(add_indicator=True, fill_value=0, strategy='constant'), pca=KernelPCA(), pca__n_components=5, scaler=RobustScaler(), selector=SelectKBest(), selector__k=5


/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 962  963  964  965  966  967  980  981 1951 1952 1953 1954 1955 1956
 1969 1970] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:55:31] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/5; 1/9] END clf=XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_delta_step=None, max_depth=5,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None), imputer=SimpleImputer(add_indicator=True,

/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 962  963  964  965  966  967  980  981 1051 1052 1053 1054 1055 1056
 1069 1070] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:56:26] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 2/5; 1/9] END clf=XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_delta_step=None, max_depth=5,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None), imputer=SimpleImputer(add_indicator=True,

...

[08:21:28] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 5/5; 5/9] END clf=XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_delta_step=None, max_depth=5,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None), imputer=SimpleImputer(add_indicator=True,

/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 962  963  964  965  966  967  980  981 1951 1952 1953 1954 1955 1956
 1969 1970] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:29:56] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/5; 6/9] END clf=XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_delta_step=None, max_depth=5,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None), imputer=SimpleImputer(add_indicator=True,

/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 962  963  964  965  966  967  980  981 1051 1052 1053 1054 1055 1056
 1069 1070] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:40:47] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 2/5; 6/9] END clf=XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_delta_step=None, max_depth=5,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None), imputer=SimpleImputer(add_indicator=True,

In [43]:
grid.best_estimator_

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', RobustScaler()),
                ('selector', SelectKBest(k=100)),
                ('pca', KernelPCA(n_components=50)),
                ('clf',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, enable_categorical=False,
                               gamma=0, gpu_id=-1, importance_type=None,
                               interaction_constraints='', learning_rate=0.05,
                               max_delta_step=0, max_depth=7,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=500,
                               n_jobs=4, num_parallel_tree=1,
                               objective='multi:softprob', predictor='auto',
                               random_state=0, reg_alpha=0, reg_lambda=1,
                  

In [54]:
# total fit time in min
grid.cv_results_['mean_fit_time'].sum()/60+grid.refit_time_/60

47.13453327814737

In [49]:
grid.cv_results_

{'mean_fit_time': array([355.32276225, 352.91089821, 360.78657513, 446.23621502,
        542.56032243, 492.97693124]),
 'std_fit_time': array([ 7.37814669,  2.24550966,  3.57112282,  3.5199128 ,  2.80051938,
        35.66951078]),
 'mean_score_time': array([0.36865253, 0.2251111 , 0.27229733, 0.24745607, 0.3048562 ,
        0.29407144]),
 'std_score_time': array([0.15649779, 0.02188388, 0.03999984, 0.02521694, 0.03070745,
        0.02930795]),
 'param_clf': masked_array(data=[XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
                                  colsample_bynode=None, colsample_bytree=None,
                                  enable_categorical=False, gamma=None, gpu_id=None,
                                  importance_type=None, interaction_constraints=None,
                                  learning_rate=0.05, max_delta_step=None, max_depth=7,
                                  min_child_weight=None, missing=nan, monotone_constraints=None,
              

In [64]:
grid.best_estimator_

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', RobustScaler()),
                ('selector', SelectKBest(k=100)),
                ('pca', KernelPCA(n_components=50)),
                ('clf',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, enable_categorical=False,
                               gamma=0, gpu_id=-1, importance_type=None,
                               interaction_constraints='', learning_rate=0.05,
                               max_delta_step=0, max_depth=7,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=500,
                               n_jobs=4, num_parallel_tree=1,
                               objective='multi:softprob', predictor='auto',
                               random_state=0, reg_alpha=0, reg_lambda=1,
                  

In [50]:
grid.best_score_

0.7422394774473015

In [56]:
y_pred = grid.best_estimator_.predict(X_test_full)

In [72]:
file_name = 'results_first_run_partdata.csv'

In [73]:
df_res = pd.DataFrame({'id': np.arange(len(y_pred)), 'y': y_pred.ravel()})
df_res.to_csv(f'submissions/{file_name}', header=True, index=False)